<a href="https://www.kaggle.com/code/arieswang98/titanic-ml-02?scriptVersionId=113172895" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np 
import pandas as pd
import warnings

import seaborn as sns
import plotly
import plotly.express as px
from matplotlib import pyplot as plt
import scikitplot as skplt

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import xgboost as xgb

import pytorch_lightning as pl

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optimizer

In [ ]:
# Read data 
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

# basic info
print('\n'+'Some basic information about the train dataset'+'\n')
train.info()

print('\n'+'Some basic information about the test dataset'+'\n')
test.info()

In [ ]:
# Feature engineering
## Proprecess data
train['Sex'].replace('male', 0 ,inplace= True)
train['Sex'].replace('female', 1 ,inplace= True)
train['Relation'] = train['SibSp']+train['Parch']
train['Ifhavekid'] = train['Parch']
train['Ifhavekid'][train['Ifhavekid']>0] = 1

dfresult = pd.DataFrame()
df = pd.get_dummies(train['Embarked'])
df.columns = ['Embarked_' +str(x) for x in df.columns ]
train = pd.concat([train,df],axis = 1)

train.Age = (train.Age-min(train.Age))/(max(train.Age)-min(train.Age))
train.Fare = (train.Fare-min(train.Fare))/(max(train.Fare)-min(train.Fare))

test['Sex'].replace('male', 0 ,inplace= True)
test['Sex'].replace('female', 1 ,inplace= True)
test['Relation'] = test['SibSp']+test['Parch']
test['Ifhavekid'] = test['Parch']
test['Ifhavekid'][test['Ifhavekid']>0] = 1

dfresult = pd.DataFrame()
df = pd.get_dummies(test['Embarked'])
df.columns = ['Embarked_' +str(x) for x in df.columns ]
test = pd.concat([test,df],axis = 1)

test.Age = (test.Age-min(test.Age))/(max(test.Age)-min(test.Age))
test.Fare = (test.Fare-min(test.Fare))/(max(test.Fare)-min(test.Fare))

In [ ]:
X = pd.concat([train['Pclass'],
               train['Sex'], train['Age'], train['SibSp'], train['Parch'],  
               train['Relation'], train['Fare']],axis=1)
for column in X:
    if np.any(X[column].isnull()):
        mean_val = np.mean(X[column])
        X[column].fillna(mean_val, inplace=True)

y = train['Survived']

In [ ]:
X.describe()

In [ ]:
# reference：https://blog.csdn.net/yuekangwei/article/details/110310970

import numpy as np
import torch
from collections import Counter
from sklearn import datasets
import torch.nn.functional as Fun

# 数据准备
X = np.array(X)
y = np.array(y)
input=torch.FloatTensor(X)
label=torch.LongTensor(y)

# 定义BP神经网络
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # 定义隐藏层网络
        self.out = torch.nn.Linear(n_hidden, n_output)   # 定义输出层网络

    def forward(self, x):
        x = Fun.relu(self.hidden(x))      # 隐藏层的激活函数
        x = self.out(x)                   # 输出层不用激活函数
        return x

# 定义优化器损失函数
net = Net(n_feature=7, n_hidden=200, n_output=2)    #n_feature:输入的特征维度,n_hiddenb:神经元个数,n_output:输出的类别个数
optimizer = torch.optim.SGD(net.parameters(), lr=0.07) # 优化器选用随机梯度下降方式
loss_func = torch.nn.CrossEntropyLoss() # 对于多分类一般采用的交叉熵损失函数,


# 训练数据
trainloss = []
trainacc = []
for t in range(200):
    out = net(input)                 # 输入input,输出out
    loss = loss_func(out, label)     # 输出与label对比
    
    trainloss.append(loss.item())
    
    prediction = torch.max(out, 1)[1] # 返回index  0返回原值
    pred_y = prediction.data.numpy()
    target_y = label.data.numpy()
    accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
    trainacc.append(accuracy)
    
    optimizer.zero_grad()   # 梯度清零
    loss.backward()         # 前馈操作
    optimizer.step()        # 使用梯度优化器


# 绘制训练过程
plt.title('Train Loss')
plt.plot(np.arange(len(trainloss)), trainloss)
plt.legend(['Train Loss'], loc='upper right')
plt.show()

plt.title('Train Accuracy')
plt.plot(np.arange(len(trainacc)), trainacc)
plt.legend(['Train Accuracy'], loc='upper right')
plt.show()

# 得出结果
out = net(input) #out是一个计算矩阵，可以用Fun.softmax(out)转化为概率矩阵
prediction = torch.max(out, 1)[1] # 返回index  0返回原值
pred_y = prediction.data.numpy()
target_y = label.data.numpy()

# 衡量准确率
accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
print("预测准确率",accuracy)





In [ ]:
# Predict
data_to_pred = pd.concat([test['Pclass'], test['Sex'], test['Age'], 
                          test['SibSp'], test['Parch'], test['Relation'], 
                          test['Fare']],axis=1)
for column in data_to_pred:
    if np.any(data_to_pred[column].isnull()):
        mean_val = np.mean(data_to_pred[column])
        data_to_pred[column].fillna(mean_val, inplace=True)

data_to_pred = np.array(data_to_pred)
input = torch.FloatTensor(data_to_pred)
out = net(input) #out是一个计算矩阵，可以用Fun.softmax(out)转化为概率矩阵
prediction = torch.max(out, 1)[1] # 返回index  0返回原值
pred_y = prediction.data.numpy()

res = pd.concat([test['PassengerId'], pd.Series(pred_y)], axis=1)
res.columns = ['PassengerId', 'Survived']
res.to_csv('/kaggle/working/submission.csv', index=False)
print('over')